In [1]:
import numpy as np
import torch
from sklearn import preprocessing

In [2]:
gbin1 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin1_all.npz')
pbin1 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin1_all.npz')

gbin2 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin2_all.npz')
pbin2 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin2_all.npz')

gbin3 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin3_all.npz')
pbin3 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin3_all.npz')

gbin4 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin4_all.npz')
pbin4 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin4_all.npz')

gbin5 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin5_all.npz')
pbin5 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin5_all.npz')

gbin6 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin6_all.npz')
pbin6 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin6_all.npz')

gbin7 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin7_all.npz')
pbin7 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin7_all.npz')

gbin8 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin8_all.npz')
pbin8 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin8_all.npz')

gbin9 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/gamma_energy_bin9_all.npz')
pbin9 = np.load('/home/jua/corsika/run/energy_var/npz/square/0522/bin/all/proton_energy_bin9_all.npz')

In [20]:
def norm(Ebin, g, p, normtype):
    g_label = torch.zeros([len(g)])
    p_label = torch.ones([len(p)])
    if normtype == "minmax":
        min_max_scaler = preprocessing.MinMaxScaler()
        for j in range(len(g)):
            for i in range(4):
                g[j][i] = min_max_scaler.fit_transform(g[j][i])
        for j in range(len(p)):
            for i in range(4):
                p[j][i] = min_max_scaler.fit_transform(p[j][i])
    elif normtype == "std":
        for j in range(len(g)):
            for i in range(4):
                scaler = preprocessing.StandardScaler().fit(g[j][i])
                g[j][i] = scaler.transform(g[j][i])
        for j in range(len(p)):
            for i in range(4):
                scaler = preprocessing.StandardScaler().fit(p[j][i])
                p[j][i] = scaler.transform(p[j][i])
    images = np.vstack((g,p))
    labels = np.hstack((g_label, p_label))
    np.savez_compressed("/home/jua/SWGO/square/data/{}/bin{}.npz".format(normtype, Ebin),image=images,label=labels)
    return len(g), len(p)

In [29]:
norm(9,gbin9['image'][:],pbin9['image'][:],'std')

(19, 24)